<a href="https://colab.research.google.com/github/kingajutrzenka/scikit-learn/blob/main/metryki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import bibliotek

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import sklearn as sk

# Metryki - klasyfikacja binarna

**accuracy (dokładność klasyfikacji) = (correct predictions/ total predictions) *100%**

In [3]:
y_true = np.array([1, 0, 1, 1, 0, 1, 0, 1, 1, 0]) # oznaczenie klas prawdziwych
y_pred = np.array([0, 1, 1, 1, 0, 0, 1, 0, 1, 1]) # oznaczenie klas predykowanych

# można sobie porównać odpowiedzi

In [4]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)
# pokazuje poprawność tutaj 40%

0.4

In [5]:
results = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred}) #łatwiej nam teraz na to patrzyć
results

,y_true,y_pred
0,1,0
1,0,1
2,1,1
3,1,1
4,0,0
5,1,0
6,0,1
7,1,0
8,1,1
9,0,1


In [6]:
results = results.sort_values(by='y_true')
results = results.reset_index(drop=True)
results['sample'] = results.index + 1
results

,y_true,y_pred,sample
0,0,1,1
1,0,0,2
2,0,1,3
3,0,1,4
4,1,1,5
5,1,1,6
6,1,0,7
7,1,0,8
8,1,0,9
9,1,1,10


In [7]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=results['sample'], y=results['y_true'], mode = 'markers', name='True'), row=1, col=1)
fig.add_trace(go.Scatter(x=results['sample'], y=results['y_pred'], mode = 'markers', name='Predicted'), row=2, col=1)
fig.update_layout(width=900, height=500, title_text='Klasyfikator binarny')
fig.show()
# mmay klasy najpierw 0, potem 1 , jakby było więcej klas to by były schodki 0, 1, 2...

In [8]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred)

In [9]:
import plotly.figure_factory as ff
def plot_confusion_matrix(cm):
    cm = cm[::-1]
    cm = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_1', 'true_0'])

    fig = ff.create_annotated_heatmap(cm.values, x=list(cm.columns), y=list(cm.index),
                                    colorscale='ice', showscale=True, reversescale=True)
    fig.update_layout(width=400, height=400, title_text='Confusion matrix', font_size = 16)
    fig.show()

plot_confusion_matrix(cm)

# po lewej wartości prawdziwe, u góry przewidziane, na przekątnej musimy mieć jak najwięcej

In [14]:
tn, fp, fn, tp = cm.ravel()
"""
tn to lg
fp to pg
fn to ld
tp to pd
cm.ravel() rozprostuje macierz
true są na przekątnej
negative 0, positive 1
"""
print(f'True negative: {tn}')
print(f'False positive: {fp}')
print(f'False negative: {fn}')
print(f'True positive: {tp}')

True negative: 1
False positive: 3
False negative: 3
True positive: 3


In [11]:
# Type I error (I - one) - False Positive Rate
fpr = fp / (fp + tn)
print(f'False positive rate: {fpr}')
# jak dużo przewidzieliśmy zdarzeń pozytywnych, gdy była klasa negatywna

False positive rate: 0.75


In [12]:
# Type II error - False Negative Rate
fnr = fn / (fn + tp)
print(f'False negative rate: {fnr}')
# jak dużo przewidzieliśmy zdarzeń negatywnych, gdy była klasa pozytywna
# bardziej niebezpieczny np. nie wykrycie przestępców
# musimy go minimalizować

False negative rate: 0.5


In [13]:
# precision - ile obserwacji przewidywanych jako pozytywne są pozytywne
precision = tp / (tp + fp)
print(f'Precision: {precision}')

Precision: 0.5


In [15]:
# reacall - jak wiele obserwacji z wszystkich pozytywnych sklasyfikowaliśmy jako pozytywne
recall = tp / (tp + fn)
print(f'Recall: {recall}')

Recall: 0.5


In [19]:
from sklearn.metrics import roc_curve

fpr, tpr, thresh = roc_curve(y_true, y_pred, pos_label=1) #post_label = 1 klasa pozytywna
roc = pd.DataFrame({'fpr': fpr, 'tpr': tpr})
roc


,fpr,tpr
0,0.00,0.0
1,0.75,0.5
2,1.00,1.0


In [22]:
def plot_roc_curve(y_true, y_pred):
    #Binary classification
    from sklearn.metrics import roc_curve
    fpr, tpr, thresh = roc_curve(y_true, y_pred, pos_label=1)

    fig = go.Figure(data=[go.Scatter(x=fpr, y=tpr, line_color='red', name='ROC Curve'),
                        go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line_dash="dash", line_color="navy")],
                    layout=go.Layout(xaxis_title='False Positive Rate',
                                     yaxis_title='True Positive Rate',
                                     title='ROC Curve',
                                     showlegend=False,
                                     width=800))

    fig.show()

plot_roc_curve(y_true, y_pred)

# Metryki - klasyfikacja wieloklasowa

In [24]:
y_true = np.array([1, 0, 2, 1, 0, 1, 0, 1, 1, 0]) # oznaczenie klas prawdziwych
y_pred = np.array([0, 1, 1, 1, 0, 0, 1, 0, 1, 2])

In [25]:
accuracy_score(y_true, y_pred)

0.3

In [27]:
cm = confusion_matrix(y_true, y_pred)
cm

array([[1, 2, 1],
       [3, 2, 0],
       [0, 1, 0]])

In [28]:
import plotly.figure_factory as ff
def plot_confusion_matrix(cm):
    cm = cm[::-1]
    cm = pd.DataFrame(cm, columns=['pred_0', 'pred_1', 'pred_2'], index=['true_2', 'true_1', 'true_0']) # tylko dodajemy klase

    fig = ff.create_annotated_heatmap(cm.values, x=list(cm.columns), y=list(cm.index),
                                    colorscale='ice', showscale=True, reversescale=True)
    fig.update_layout(width=400, height=400, title_text='Confusion matrix', font_size = 16)
    fig.show()

plot_confusion_matrix(cm)

# po lewej wartości prawdziwe, u góry przewidziane, na przekątnej musimy mieć jak najwięcej

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.25      0.25      0.25         4
           1       0.40      0.40      0.40         5
           2       0.00      0.00      0.00         1

    accuracy                           0.30        10
   macro avg       0.22      0.22      0.22        10
weighted avg       0.30      0.30      0.30        10



In [31]:
print(classification_report(y_true, y_pred, target_names=['label_1', 'label_2', 'label_3']))

              precision    recall  f1-score   support

     label_1       0.25      0.25      0.25         4
     label_2       0.40      0.40      0.40         5
     label_3       0.00      0.00      0.00         1

    accuracy                           0.30        10
   macro avg       0.22      0.22      0.22        10
weighted avg       0.30      0.30      0.30        10

